## Load Packages and Data

In [2]:
# Load Packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Load data from google drive
url = 'https://docs.google.com/spreadsheets/d/1Kz8fO-V7h5dwXl-hty1CNKybOttZnv3A/export?format=csv&gid=377439479'
df_demographics = pd.read_csv(url)

df_demographics.head()

,Local ID,Date Enrolled in START,Status,Status Date,Time Enrolled in START,Source of referral to START,Suitability of enrollment in START,Individual/Caregiver reliable access to technology,Presenting problems at time of enrollment,Services at Enrollment,...,State sponsored ICF/DD stay in past 1 - 5 years (at enrollment to START),Jailed in the past year (at enrollment to START),Jailed in the past 1 - 5 years (at enrollment to START),Employment Status at Enrollment,Grade In School,Current IEP,Current 504 Plan,School Setting,Custody Status,Child Lives With
0,11128335,7/1/2010,Active,7/22/2010,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"Laptop/desktop, Tablet, Telephone landline","Family needs assistance, Self-injurious","Behavioral support services, Case management/s...",...,No,No,No,Day/vocational program,NaN,No,No,NaN,NaN,NaN
1,11134104,11/1/2011,Active,11/30/2011,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"Cell phone service, High speed internet, Smart...","Aggression (physical, verbal, property destruc...","Behavioral support services, Case management/s...",...,No,No,No,Volunteer,NaN,No,No,NaN,NaN,NaN
2,11138520,5/1/2012,Active,5/21/2012,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"High speed internet, Tablet, Telephone landline","Diagnosis and treatment plan assistance, Famil...","Case management/service coordination, Day serv...",...,No,No,No,Not employed,NaN,No,No,NaN,NaN,NaN
3,11132094,5/1/2013,Active,5/9/2013,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"Cell phone service, High speed internet, Lapto...","Aggression (physical, verbal, property destruc...","Behavioral support services, Case management/s...",...,No,No,No,Employment (PT),NaN,No,No,NaN,NaN,NaN
4,975134C,7/1/2014,Active,7/1/2014,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"High speed internet, Laptop/desktop, Smartphon...","Aggression (physical, verbal, property destruc...","Case management/service coordination, Mental h...",...,No,No,No,Not employed,NaN,No,No,NaN,NaN,NaN


## Individual/Caregiver Reliable Access to Technology Variable Cleaning

In [64]:
df_demographics['tech_access'] = df_demographics['Individual/Caregiver reliable access to technology']
df_demographics['tech_access']

0              Laptop/desktop, Tablet, Telephone landline
1       Cell phone service, High speed internet, Smart...
2         High speed internet, Tablet, Telephone landline
3       Cell phone service, High speed internet, Lapto...
4       High speed internet, Laptop/desktop, Smartphon...
                              ...                        
4981                                                  NaN
4982                                                  NaN
4983                                   Telephone landline
4984                                                  NaN
4985                                   Cell phone service
Name: tech_access, Length: 4986, dtype: object

In [65]:
def clean_tech_access(entry):
    if isinstance(entry, str):  # Check if the entry is a string
        items = entry.split(',')
        items = [item.strip().lower() for item in items]
        items = sorted(set(items))  # Remove duplicates and sort
        return ', '.join(items)
    else:
        return entry  # Return the entry as is if it's not a string

In [66]:
df_demographics['tech_access'] = df_demographics['tech_access'].apply(clean_tech_access)
df_demographics['tech_access'] = df_demographics['tech_access'].fillna('none')
df_demographics['tech_access']

0              laptop/desktop, tablet, telephone landline
1       cell phone service, high speed internet, smart...
2         high speed internet, tablet, telephone landline
3       cell phone service, high speed internet, lapto...
4       high speed internet, laptop/desktop, smartphon...
                              ...                        
4981                                                 none
4982                                                 none
4983                                   telephone landline
4984                                                 none
4985                                   cell phone service
Name: tech_access, Length: 4986, dtype: object

In [67]:

#df_demographics[df_demographics['tech_access'].str.contains('other')]
df_demographics.loc[df_demographics['tech_access'].str.contains('other'), 'tech_access']

2564    other: has a cell phone but it does not work.,...
3784    cell phone service, other: individual prefers ...
4337    cell phone service, high speed internet, lapto...
4462    other: high internet risk - there is no wifi o...
4757    high speed internet, other: communication devi...
4918          cell phone service, other: zoom, smartphone
4974    cell phone service, high speed internet, lapto...
Name: tech_access, dtype: object

In [75]:
# Clean the 'others'
# Define conditions and replacements
conditions = [
    df_demographics['tech_access'].str.contains("other: has a cell phone but it does not work", case=False),
    df_demographics['tech_access'].str.contains("cell phone service, other: individual prefers", case=False),
    df_demographics['tech_access'].str.contains("other: high internet risk - there is no wifi o", case=False),
    df_demographics['tech_access'].str.contains("high speed internet, other: communication devi", case=False),
    df_demographics['tech_access'].str.contains("cell phone service, other: zoom, smartphone", case=False),
    df_demographics['tech_access'].str.contains("cell phone service, high speed internet, laptop/desktop, other: computer", case=False),
    df_demographics['tech_access'].str.contains("cell phone service, high speed internet, laptop/desktop, other: individual", case=False)
]
replacements = [
    "telephone landline",
    "cell phone service, telephone landline",
    "none",
    "high speed internet, telephone landline",
    "cell phone service, smartphone",
    "cell phone service, high speed internet,laptop/desktop, smartphone, tablet",
    "cell phone service, high speed internet, laptop/desktop, telephone landline"
]

# Apply replacements based on conditions
df_demographics['tech_access'] = np.select(conditions, replacements, df_demographics['tech_access'])
df_demographics['tech_access']

0              laptop/desktop, tablet, telephone landline
1       cell phone service, high speed internet, smart...
2         high speed internet, tablet, telephone landline
3       cell phone service, high speed internet, lapto...
4       high speed internet, laptop/desktop, smartphon...
                              ...                        
4981                                                 none
4982                                                 none
4983                                   telephone landline
4984                                                 none
4985                                   cell phone service
Name: tech_access, Length: 4986, dtype: object

In [77]:
df_demographics['num_categories'] = df_demographics['tech_access'].apply(lambda x: len(x.split(', ')))

# Calculate the average number of categories
average_num_categories = df_demographics['num_categories'].mean()

print("Average number of categories per entry:", average_num_categories) # maybe going forward in analysis we can make a dummy if people have 2 or more, etc

Average number of categories per entry: 1.4249899719213799


In [ ]:
# One-Hot Encode the tech_access column
# dummies = df_demographics['tech_access'].str.get_dummies(sep=', ')

# Concatenate the one-hot encoded columns with the original DataFrame
# df_tech_access = pd.concat([df_demographics['Local ID'], dummies], axis=1)
# df_tech_access

## Jail Variable Cleaning

In [109]:
df_demographics['jailed_prev_year'] = df_demographics['Jailed in the past year (at enrollment to START)']
df_demographics['jailed_prev_year']

0       No
1       No
2       No
3       No
4       No
        ..
4981    No
4982    No
4983    No
4984    No
4985    No
Name: jailed_prev_year, Length: 4986, dtype: object

In [111]:
df_demographics['jailed_prev_year'].value_counts()

jailed_prev_year
No     4871
Yes     115
Name: count, dtype: int64

In [112]:
df_demographics['jailed_prev_five_years'] = df_demographics['Jailed in the past 1 - 5 years (at enrollment to START)']
df_demographics['jailed_prev_five_years']

0       No
1       No
2       No
3       No
4       No
        ..
4981    No
4982    No
4983    No
4984    No
4985    No
Name: jailed_prev_five_years, Length: 4986, dtype: object

In [113]:
df_demographics['jailed_prev_five_years'].value_counts()

jailed_prev_five_years
No     4857
Yes     129
Name: count, dtype: int64

In [121]:
def combine_jail_history(df):
    # Create a new variable indicating previous jail history
    df['jailed_prev'] = df.apply(lambda row: 'Yes' if row['jailed_prev_year'] == 'Yes' 
                                                    or row['jailed_prev_five_years'] == 'Yes'
                                               else 'No', axis=1)
    
    return df

# call func
combine_jail_history(df_demographics)
df_demographics


,Local ID,Date Enrolled in START,Status,Status Date,Time Enrolled in START,Source of referral to START,Suitability of enrollment in START,Individual/Caregiver reliable access to technology,Presenting problems at time of enrollment,Services at Enrollment,...,Employment Status at Enrollment,Grade In School,Current IEP,Current 504 Plan,School Setting,Custody Status,Child Lives With,jailed_prev_year,jailed_prev_five_years,jailed_prev
0,11128335,7/1/2010,Active,7/22/2010,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"Laptop/desktop, Tablet, Telephone landline","Family needs assistance, Self-injurious","Behavioral support services, Case management/s...",...,Day/vocational program,NaN,No,No,NaN,NaN,NaN,No,No,No
1,11134104,11/1/2011,Active,11/30/2011,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"Cell phone service, High speed internet, Smart...","Aggression (physical, verbal, property destruc...","Behavioral support services, Case management/s...",...,Volunteer,NaN,No,No,NaN,NaN,NaN,No,No,No
2,11138520,5/1/2012,Active,5/21/2012,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"High speed internet, Tablet, Telephone landline","Diagnosis and treatment plan assistance, Famil...","Case management/service coordination, Day serv...",...,Not employed,NaN,No,No,NaN,NaN,NaN,No,No,No
3,11132094,5/1/2013,Active,5/9/2013,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"Cell phone service, High speed internet, Lapto...","Aggression (physical, verbal, property destruc...","Behavioral support services, Case management/s...",...,Employment (PT),NaN,No,No,NaN,NaN,NaN,No,No,No
4,975134C,7/1/2014,Active,7/1/2014,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,"High speed internet, Laptop/desktop, Smartphon...","Aggression (physical, verbal, property destruc...","Case management/service coordination, Mental h...",...,Not employed,NaN,No,No,NaN,NaN,NaN,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4981,64995,8/1/2019,Inactive (other):,12/16/2021,Business Hours (Monday - Friday 8am - 5pm),Residential provider - community,Appropriate,NaN,"Aggression (physical, verbal, property destruc...","Behavioral support services, Case management/s...",...,Day/vocational program,Graduated,No,No,NaN,Parental custody,Residential staff,No,No,No
4982,366747,3/1/2019,Inactive (stable functioning),12/16/2021,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,NaN,"Aggression (physical, verbal, property destruc...","Behavioral support services, Case management/s...",...,Day/vocational program,NaN,NaN,NaN,NaN,NaN,NaN,No,No,No
4983,40366,10/1/2018,Inactive (stable functioning),12/16/2021,Business Hours (Monday - Friday 8am - 5pm),Emergency services/mobile crisis team,Appropriate,Telephone landline,"Aggression (physical, verbal, property destruc...","Case management/service coordination, Day serv...",...,Day/vocational program,NaN,No,No,NaN,NaN,NaN,No,No,No
4984,11150993,11/1/2013,Active,12/16/2021,Business Hours (Monday - Friday 8am - 5pm),Case Manager/Service Coordinator,Appropriate,NaN,"Family needs assistance, Other: Team requestin...","Case management/service coordination, Day serv...",...,Day/vocational program,NaN,No,No,NaN,NaN,NaN,No,No,No


In [125]:
df_demographics['jailed_prev'].value_counts()

jailed_prev
No     4796
Yes     190
Name: count, dtype: int64

In [8]:
df_demographics["Who is the individual's primary caregiver?"].value_counts()

Who is the individual's primary caregiver?
Parent                       2707
Paid support staff           1292
Self                          285
Other family member           237
Guardian                      215
Authorized representative     121
Name: count, dtype: int64

In [10]:
df_demographics['Ethnicity'].value_counts()

Ethnicity
Not of Hispanic origin                      3798
Hispanic - specific origin not specified     777
Unknown, not collected                       411
Name: count, dtype: int64

In [13]:


# new labels
replacements = {
    'Not of Hispanic origin': 'Not Hispanic Descent',
    'Hispanic - specific origin not specified': 'Hispanic Descent',
    'Unknown, not collected': 'Unknown'
}

# replace the values in ethnicity
df_demographics['Ethnicity'] = df_demographics['Ethnicity'].replace(replacements)


df_demographics['Ethnicity'].value_counts()


Ethnicity
Not Hispanic Descent    3798
Hispanic Descent         777
Unknown                  411
Name: count, dtype: int64

In [19]:
# filter out unknowns
df_demographics['ethnicity_binary'] = df_demographics['Ethnicity'].apply(lambda x: x if x != 'Unknown' else None)
df_demographics['ethnicity_binary'].value_counts()

ethnicity_binary
Not Hispanic Descent    3798
Hispanic Descent         777
Name: count, dtype: int64